In [ ]:
#Import packages

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import IPython
import sklearn #Machine Learning package
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Import Dataset

df = pd.read_csv ('Peak_matching_Campylobacter.csv') #Import the peak matching table as a dataframe
Y = df['AMR'] # Target variable will be the AMR pattern (what you want to predict)
X = df.drop(columns=['AMR','ID']) #MALDI peaks and their intensities are feature set (The AMR and ID columns are discarded)
saved_cols = X.columns #store columns names in this variable

df

In [ ]:
#Features transformation/Scaling

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() #Store the function in the scaler variable
scaled = scaler.fit_transform(X) #transformed values into the (0,1) range

scaled_df = pd.DataFrame(scaled, Y, columns = saved_cols) #Create a new dataframe with the scaled intensities
scaled_df = scaled_df.reset_index() 

scaled_df.loc[scaled_df['AMR'].str.contains('Cip'), 'AMR'] = '1' # it will transform all label containing the string "Cip" into 1 (1 = resistant)
scaled_df.loc[~scaled_df['AMR'].str.contains('1'), 'AMR'] = '0' # it will transform all label NOT containing the string "1" into 0 (0 = susceptible)

scaled_df

In [ ]:
Y = scaled_df['AMR'].astype(float) # Need to convert categorical to numerical type value
X = scaled_df.drop(columns=['AMR']) 

print(scaled_df.groupby('AMR').size()) # A total of 183 CipR and 157 susceptible strains

In [ ]:
#Features selection

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score

clf = RandomForestClassifier(random_state=10, n_estimators=300) 
clf = clf.fit(X, Y)

model = SelectFromModel(clf, prefit=True)
X_new  = model.transform(X)

print("Before Feature selection",X.shape) # Before and After the features selection
print("After Feature selection",X_new.shape)

scaled_features_df = pd.DataFrame(X_new, Y) #Create a new dataframe with the scaled intensities

features = X.columns.values
print(features[model.get_support()]) # Features of interest retained for the model

In [ ]:
#Training/Validation/Test set

from sklearn.model_selection import train_test_split

X_train, X_rem, Y_train, Y_rem = train_test_split(X_new, Y, test_size=0.3,stratify=Y,random_state=0, shuffle=True) # Split the dataset into a 70/30 training/test set based on the AMR label

X_valid, X_test, Y_valid, Y_test = train_test_split(X_rem,Y_rem, test_size=0.5)

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)


In [ ]:
#Model selection

from sklearn.metrics import plot_roc_curve, roc_curve, auc, precision_recall_curve, plot_precision_recall_curve, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

#Fitting different models to the training data

LR = LogisticRegression().fit(X_train, Y_train) 
NB = GaussianNB().fit (X_train, Y_train)
RF = RandomForestClassifier().fit(X_train,Y_train)

#ROC Curve

classifiers = [LR, NB, RF]
ax = plt.gca()
for i in classifiers:
    plot_roc_curve(i, X_valid, Y_valid, ax=ax)
    plt.plot([0,1], [0,1], ':',linewidth=1.4, c='k')
    plt.legend(bbox_to_anchor = (1.05, 0.6))
    

In [ ]:
#Precision-Recall curve

classifiers1 = [LR, NB, RF]
ax = plt.gca()
for i in classifiers1 :
    plot_precision_recall_curve(i, X_valid, Y_valid, ax=ax)
    plt.legend(bbox_to_anchor = (1.8, 0.6))
    
for z in classifiers1 :
    precision, recall, thresholds = precision_recall_curve(Y_test, z.predict_proba(X_test)[:,1])
    area = auc(recall, precision)
    print("AUPRC",area)    


In [ ]:
#Tune the model: Grid Search

from sklearn.model_selection import GridSearchCV

# Design the grid of parameters you want to investigate

param_grid = {
'max_depth': [80, 90, 100, 110],
'max_features': [2, 3],
'min_samples_leaf': [3, 4, 5],
'min_samples_split': [8, 10, 12],
'n_estimators': [100, 200, 300, 1000]
}

RF = RandomForestClassifier()
grid_search = GridSearchCV(estimator = RF, param_grid = param_grid, cv= 10, n_jobs = -1, verbose = 2, scoring='f1') # Best parameters for the Random Forest model based on the F1 score
grid_search.fit(X_valid, Y_valid)
grid_search.best_estimator_

In [ ]:
#Performance

RF = RandomForestClassifier(max_depth=110, max_features=2, min_samples_leaf=3,min_samples_split=8, n_estimators=200).fit (X_train, Y_train) # Fit the model with the optimized hyperparameters

#ROC Curve

classifiers = [RF]
ax = plt.gca()
for i in classifiers:
    plot_roc_curve(i, X_valid, Y_valid, ax=ax)
    plt.plot([0,1], [0,1], ':',linewidth=1.4, c='k')
    plt.legend(bbox_to_anchor = (1.05, 0.6))

In [ ]:
#Precision-Recall curve

classifiers1 = [RF]
ax = plt.gca()
for i in classifiers1 :
    plot_precision_recall_curve(i, X_valid, Y_valid, ax=ax)
    plt.legend(bbox_to_anchor = (1.8, 0.6))
    
# Compute Precision-Recall and plot curve

precision, recall, thresholds = precision_recall_curve(Y_test, RF.predict_proba(X_test)[:,1])
area = auc(recall, precision)

print("AUPRC",area)

In [ ]:
#Optimization of the Model

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

#Metrics

y_pred_rf=RF.predict(X_valid)
confusion = confusion_matrix(Y_valid,y_pred_rf)
disp = ConfusionMatrixDisplay(confusion_matrix=confusion, display_labels=RF.classes_).plot()
plt.tick_params(axis=u'both', which=u'both',length=0)

precision= precision_score(Y_valid, y_pred_rf)
recall=recall_score(Y_valid, y_pred_rf)
F1= f1_score(Y_valid, y_pred_rf)
print('F1-score:',F1)
print('Precision:',precision)
print('Recall:',recall)

total1=sum(sum(confusion))
accuracy1=(confusion[0,0]+confusion[1,1])/total1
print ('Accuracy : ', accuracy1)
specificity1 = confusion[0,0]/(confusion[0,0]+confusion[0,1])
print('Specificity : ', specificity1 )
sensitivity1 = confusion[1,1]/(confusion[1,0]+confusion[1,1])
print('Sensitivity : ', sensitivity1)

In [ ]:
#Tune the model: Best threshold

from numpy import arange
from numpy import argmax
from matplotlib import pyplot
from numpy import sqrt

# predict probabilities
yhat = RF.predict_proba(X_valid)

# keep probabilities for the positive outcome only
yhat = yhat[:, 1]

# calculate roc curves
precision, recall, thresholds = precision_recall_curve(Y_valid, yhat)

# convert to f score
fscore = (2 * precision * recall) / (precision + recall)

# locate the index of the largest f score
ix = argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))

# plot the roc curve for the model
no_skill = len(Y_valid[Y_valid==1]) / len(Y_valid)
pyplot.plot([0,1], [no_skill,no_skill], linestyle='--', label='No Skill')
pyplot.plot(recall, precision, marker='.', label='Logistic')
pyplot.scatter(recall[ix], precision[ix], marker='o', color='black', label='Best')

# axis labels
pyplot.xlabel('Recall')
pyplot.ylabel('Precision')
pyplot.legend()

# show the plot
pyplot.show()

In [ ]:
#Set the new threshold

y_pred_new_threshold = (RF.predict_proba(X_valid)[:,1]>=0.337941).astype(int)

#Metrics

confusion = confusion_matrix(Y_valid,y_pred_new_threshold )
disp = ConfusionMatrixDisplay(confusion_matrix=confusion, display_labels=RF.classes_).plot()
plt.tick_params(axis=u'both', which=u'both',length=0)
precision= precision_score(Y_valid, y_pred_new_threshold)
recall=recall_score(Y_valid, y_pred_new_threshold)
F1= f1_score(Y_valid, y_pred_new_threshold)
print('F1-score:',F1)
print('Precision/PPV:',precision)
print('Recall:',recall)


total1=sum(sum(confusion))
accuracy1=(confusion[0,0]+confusion[1,1])/total1
print ('Accuracy : ', accuracy1)
specificity1 = confusion[0,0]/(confusion[0,0]+confusion[0,1])
print('Specificity : ', specificity1 )
sensitivity1 = confusion[1,1]/(confusion[1,0]+confusion[1,1])
print('Sensitivity : ', sensitivity1)
NPV = confusion[0,0]/(confusion[0,0]+confusion[1,0])
print('NPV: ', NPV)

In [ ]:
#Evaluation of the tuned model on the test set

y_pred_new_threshold = (RF.predict_proba(X_test)[:,1]>=0.337941).astype(int)

#Metrics

confusion = confusion_matrix(Y_test,y_pred_new_threshold )
disp = ConfusionMatrixDisplay(confusion_matrix=confusion, display_labels=RF.classes_).plot()
plt.tick_params(axis=u'both', which=u'both',length=0)
precision= precision_score(Y_test, y_pred_new_threshold)
recall=recall_score(Y_test, y_pred_new_threshold)
F1= f1_score(Y_test, y_pred_new_threshold)
print('F1-score:',F1)
print('Precision/PPV:',precision)
print('Recall:',recall)


total1=sum(sum(confusion))
accuracy1=(confusion[0,0]+confusion[1,1])/total1
print ('Accuracy : ', accuracy1)
specificity1 = confusion[0,0]/(confusion[0,0]+confusion[0,1])
print('Specificity : ', specificity1 )
sensitivity1 = confusion[1,1]/(confusion[1,0]+confusion[1,1])
print('Sensitivity : ', sensitivity1)
NPV = confusion[0,0]/(confusion[0,0]+confusion[1,0])
print('NPV: ', NPV)



In [ ]:
#Prediction

Unknown = pd.read_csv ('List_of_Unknow_Spectra.csv') # import the unknow spectra you want classify (they need to have the same features than the one used for the training)
prediction = (RF.predict_proba(Unknown)[:,1]>=0.359180).astype(int) #Will predict the class (0 or 1)

print("Prediction",prediction)